In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd './drive/MyDrive/gaze_estimation'

In [ ]:
# Import Libraries
import cv2
import os
import pandas as pd
import numpy as np
import zipfile
import random
random.seed(43)
import warnings
warnings.filterwarnings("ignore")
from glob import glob
from tqdm.auto import tqdm

# Get Subset Information

In [ ]:
sub_indice = np.load("./ut_sub_indices_str.npy")

## Utils

In [ ]:
def convert_pose(vector: np.ndarray) -> np.ndarray:
    rot = cv2.Rodrigues(np.array(vector).astype(np.float32))[0]
    vec = rot[:, 2]
    pitch = np.arcsin(vec[1])
    yaw = np.arctan2(vec[0], vec[2])
    return np.array([pitch, yaw]).astype(np.float32)

def convert_gaze(vector: np.ndarray) -> np.ndarray:
    x, y, z = vector
    pitch = np.arcsin(-y)
    yaw = np.arctan2(-x, -z)
    return np.array([pitch, yaw]).astype(np.float32)

## Get Sub Sampling Data

In [ ]:
ut_path = "../UTMultiview"

datas = []
for s_idx in tqdm(range(sub_indice.shape[0])):
    if s_idx >= 0 and s_idx < 10:
        grp = "s00-09"
    elif s_idx >= 10 and s_idx < 20:
        grp = "s10-19"
    elif s_idx >= 20 and s_idx < 30:
        grp = "s20-29"
    elif s_idx >= 30 and s_idx < 40:
        grp = "s30-39"
    else:
        grp = "s40-49"
        
    s_id = "".join(("s", str(s_idx).zfill(2)))
    sub_synth_path = os.path.join(ut_path, grp, s_id, "synth")
    
    for d_idx in range(sub_indice.shape[1]):
        data_folder = sub_indice[s_idx][d_idx].split("/")[1]
        seq_id = data_folder.split("_")[0]
        loc_id = data_folder.split("_")[1]
        
        img_data = sub_indice[s_idx][d_idx].split("/")[2]
        img_idx = int(img_data.split(".")[0])
        
        ano_file = os.path.join(sub_synth_path, ".".join((data_folder, "csv")))
        ano_data = pd.read_csv(ano_file, header=None).iloc[:,:6]
        columns = ["gaze_x", "gaze_y", "gaze_z", "rotation_x", "rotation_y", "rotation_z"]
        ano_data.columns = columns
        
        # Head Pose Rotation Data
        pose_data = [ano_data["rotation_x"].iloc[img_idx], ano_data["rotation_y"].iloc[img_idx], ano_data["rotation_z"].iloc[img_idx]]
        # Gaze Vector Data
        gaze_data = [ano_data["gaze_x"].iloc[img_idx], ano_data["gaze_y"].iloc[img_idx], ano_data["gaze_z"].iloc[img_idx]]
        
        # Image data
        img_data_path = os.path.join(sub_synth_path, data_folder, img_data)
        image = cv2.imread(img_data_path)
        image_array = np.array(image)
        image_data_gray = cv2.cvtColor(image_array, cv2.COLOR_RGB2GRAY)
        
        if loc_id == "left":
            pose = convert_pose(pose_data)
            gaze = convert_gaze(gaze_data)
            image = image_data_gray
        else:
            image = image_data_gray[:, ::-1]
            pose = convert_pose(pose_data) * np.array([1, -1])
            gaze = convert_gaze(gaze_data) * np.array([1, -1])
        
        data_list = [s_id, seq_id, loc_id, image.ravel(), pose[0], pose[1], gaze[0], gaze[1]]
        datas.append(data_list)

ut_df = pd.DataFrame(columns=["participant_id","day","eye_location","image","head_pitch","head_yaw","gaze_pitch","gaze_yaw"], data=datas)
ut_df = ut_df.sort_values(by=['participant_id', 'day']).reset_index(drop=True)

In [ ]:
os.makedirs('../ut_dataset', exist_ok=True)
ut_df.to_parquet("../ut_dataset/sampled_utm_synth_dataset.parquet", engine='pyarrow', index=False)